# Analyze the ABC output

In [ ]:
using CSV,DataFrames,Plots,Statistics,Distributions,Measures
gr();

include("auxilliary.jl"); include("parameters.jl"); include("flow.jl"); include("abcmc.jl");
ENV["COLUMNS"]=200;

## Inputs

In [ ]:
# Dataframe of abc samples
frt = 1; lst = 10;
df = CSV.read("ABCsmp$frt.csv",DataFrame,header=false);
for id=frt+1:lst
    dftemp = CSV.read("ABCsmp$id.csv",DataFrame,header=false);
    df = hcat(df,dftemp[:,2:end],makeunique=true);
end

# Dataframe of trajectories
dftrj = CSV.read("ABCtrj$frt.csv",DataFrame,header=false);
for id=frt+1:lst
    dftemp = CSV.read("ABCtrj$id.csv",DataFrame,header=false);
    dftrj = hcat(dftrj,dftemp[:,2:end],makeunique=true);
end

# threshold for ℓerr cutoff, given as a top quantile (eg 0.05 is top 5%)
qℓ = 0.1;

## Plot formatting
For gr plot scaling was seeing<br>
10.42in = 750px* r =><br>
r = 10.42in/750px=><br>
7.5in = 750px* r *lambda = 750px*(10.42in/750px)*lambda => 7.5in/10.42in=lambda<br><br>

Also seeing fontsize 12 gets mapped to fontsize 18

In [ ]:
# Plot formatting
#  Finding that to get right output dimensions and font sizes for gr have to manually
#  compute scaling factors based on an observed print output
λplot = 100*7.5/10.42; λfont = 12/18; #scaling factors

# Final output two panel figure dimensions
figwdth = 7.45; fighght = 3.25; # inches

# Final output figure fontsizes
figtickfontsize = 12; figguidefontsize = 14; figtitlefontsize = 16;

In [ ]:
lnopt = [:solid, :dash, :dot, :dashdot, :dashdotdot];
mkopt = [:none, :auto, :circle, :rect, :star5, :diamond, :hexagon, :cross, 
         :xcross, :utriangle, :dtriangle, :rtriangle, :ltriangle, :pentagon, 
         :heptagon, :octagon, :star4, :star6, :star7, :star8, :vline, :hline, 
         :+, :x]; 

## Extract samples

In [ ]:
ncols = ncol(df)-1; nℓcols = qℓ*ncols;
println("The total number of abc sampling before conditioning: $ncols")
println("The total number of abc sampling after conditioning with top $qℓ quantile: $nℓcols")

In [ ]:
prmrg,prmvary=abcdata();prmvary[:ℓerr]=true;prmvary[:βθ]=true;
println("Varied parameters:")
for key in keys(prmvary)
    if prmvary[key]
        println(key)
    end
end

In [ ]:
# Create dictionary of row position of key parameters;
mykeys = [key for key in keys(prmvary)]; mykeys=vcat(mykeys,[:ℓerr,:βθ])
pnt=Dict{Symbol,Int64}()
for key in mykeys
    pos = 1;
    while df[pos,1]!=string(key)
        pos+=1;
    end
    pnt[key]=pos;
end;

In [ ]:
# Truncate down to trajectories that didn't return NaN
function Base.isnan(s::Union{Symbol,AbstractString})
    return false
end

dfnan = .!(isnan.(df[pnt[:ℓerr]:pnt[:ℓerr],:])); flagnan = [dfnan[1,i] for i=1:ncol(dfnan)];
df = df[:,flagnan]; dftrj = dftrj[:,flagnan];
print("The number of NaN's is "); println(sum(.!(flagnan)))

In [ ]:
# Extract marginal values conditioned on threshold
marginals = Dict{Symbol,Vector{Float64}}();
for key in keys(prmvary)
    if prmvary[key]
        marginals[key] = [v for v in df[pnt[key],2:end]];
    end
end
flag = marginals[:ℓerr] .<= quantile(marginals[:ℓerr],qℓ);
for key in keys(marginals)
    marginals[key] = marginals[key][flag]
end;

# Extract trajectories conditioned on threshold
dftrjabc = dftrj[:,2:end];
dftrjabc = dftrjabc[:,flag];
dftrjabc = hcat(DataFrame("day"=>convert(Vector,0:(nrow(dftrjabc)-1))),dftrjabc);
last(dftrjabc,7)

## Plot marginal distributions

In [ ]:
histogram(marginals[:ℓerr],title="ℓerr",labels="",size=(500,250),normalize=:pdf)

In [ ]:
histogram(marginals[:ρ],title="ρ",labels="",size=(500,250),normalize=:pdf)

In [ ]:
histogram(marginals[:αeff],title="αeff",labels="",size=(500,250),normalize=:pdf)

In [ ]:
histogram(marginals[:βα],title="βα",labels="",size=(500,250),normalize=:pdf)

In [ ]:
histogram(marginals[:γα],title="γα",labels="",size=(500,250),normalize=:pdf)

In [ ]:
histogram(marginals[:γθ],title="γθ",labels="",size=(500,250),normalize=:pdf)

In [ ]:
histogram2d(marginals[:γθ],marginals[:γα],labels="",xlabel="γθ",ylabel="γα",size=(500,250))

In [ ]:
βμ = [mean(Weibull(marginals[:βα][i],marginals[:βθ][i])) for i=1:length(marginals[:βα])];
γμ = [mean(Weibull(marginals[:γα][i],marginals[:γθ][i])) for i=1:length(marginals[:γα])];

βσ = [std(Weibull(marginals[:βα][i],marginals[:βθ][i])) for i=1:length(marginals[:βα])];
γσ = [std(Weibull(marginals[:γα][i],marginals[:γθ][i])) for i=1:length(marginals[:γα])];

histogram2d(βμ,γμ,xlabel="Length of infectious period mean",ylabel="Length of recovery period mean",size=(500,325))

In [ ]:
pltβγμ = histogram(βμ,normalize=:pdf,labels="contact interval β",linecolor=:white,alpha=0.65,linewidth=0);
histogram!(γμ,normalize=:pdf,labels="infectious period γ",linecolor=:white,alpha=0.65,linewidth=0)
plot!(xlabel="mean",ylabel="pdf",
      size=(figwdth/2*λplot,fighght*λplot),
      tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
      titlefontsize=round(figtitlefontsize*λfont))

In [ ]:
savefig("postmean.pdf")

In [ ]:
# Master figure for all posteriors
# Ex of how to use blank plots to center and vary number of subplots in a plot
#pltblank = plot(legend=false,grid=false,foreground_color_subplot=:white);
#lay = @layout [_{0.165w} a{0.335w} b{0.335w} _{0.165w};c d e];
#plot(pltblank,plt1,plt2,pltblank,plt3,plt4,plt5,layout = lay,size=(figwdth*λplot,2*fighght*λplot))

plt1 = histogram(marginals[:ρ],title="ρ",labels="",size=(500,250),normalize=:pdf,ylabel="pdf",xlabel="",
                 xticks=0.0:0.025:0.05,linecolor=:white,linewidth=0)
plt2 = histogram(βμ,title="βμ",labels="",size=(500,250),normalize=:pdf,xlabel="",
                 xticks=12.4:0.3:13.0,linecolor=:white,linewidth=0)
plt3 = histogram(βσ,title="βσ",labels="",size=(500,250),normalize=:pdf,xlabel="",linecolor=:white,linewidth=0)
plt4 = histogram(marginals[:αeff],title="αeff",labels="",size=(500,250),normalize=:pdf,ylabel="pdf",xlabel="value",
                 linecolor=:white,linewidth=0)
plt5 = histogram(γμ,title="γμ",labels="",size=(500,250),normalize=:pdf,xlabel="value",
                 xticks=12.5:1.0:15.0,linecolor=:white,linewidth=0)
plt6 = histogram(γσ,title="γσ",labels="",size=(500,250),normalize=:pdf,xlabel="value",
                 xticks=1.8:0.4:3.0,linecolor=:white,linewidth=0)

plot!(plt1,tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
           titlefontsize=round(figtitlefontsize*λfont))
plot!(plt2,tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
           titlefontsize=round(figtitlefontsize*λfont))
plot!(plt3,tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
           titlefontsize=round(figtitlefontsize*λfont))
plot!(plt4,tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
           titlefontsize=round(figtitlefontsize*λfont))
plot!(plt5,tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
           titlefontsize=round(figtitlefontsize*λfont))
plot!(plt6,tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
           titlefontsize=round(figtitlefontsize*λfont))
lay = @layout [a b c;d e f]
plot(plt1,plt2,plt3,plt4,plt5,plt6,layout = lay,size=(figwdth*λplot,2*fighght*λplot))

In [ ]:
savefig("postprm.pdf");

In [ ]:
plot([mean(Weibull(marginals[:βα][i],marginals[:βθ][i])) for i=1:length(marginals[:βα])],
     [std(Weibull(marginals[:βα][i],marginals[:βθ][i])) for i=1:length(marginals[:βα])],
     seriestype=:scatter,labels="contact interval β")
plot!([mean(Weibull(marginals[:γα][i],marginals[:γθ][i])) for i=1:length(marginals[:γα])],
     [std(Weibull(marginals[:γα][i],marginals[:γθ][i])) for i=1:length(marginals[:γα])],
     seriestype=:scatter,labels="infectiousness period γ")
plot!(xlabel="mean",ylabel="standard deviation",size=(500,325))

As a reminder on why there is a submanifold structure on the $\beta$ distribution: The boundary conditions are <br>
$y_I(0,s) = \rho f_I(s)$ and $y_I(t,0) = \left(\int^\infty_0 (y_S(t,u) + (1-\alpha(t,u))y_V(t,u))du\right)\left(\int^\infty_0\beta(u)y_I(t,u)du\right)$.<br>
If you enforce continity at $(0,0)$ between these two boundary conditions this becomes<br>
$\rho f_I(0) = y_I(0,s=0) = y_I(t=0,0) = \left(\int^\infty_0 f_S(u)du\right)\left(\int^\infty_0\beta(u)\rho f_I(u)du\right)$.<br>
If we now simplify, substitute in that $\beta(s) = \frac{\alpha}{\theta}\left(\frac{s}{\theta}\right)^{\alpha-1}$ and let $\eta=\left(\int^\infty_0 f_S(u)du\right)$ notate a constant, we obtain <br>
$f_I(0) = \frac{\eta\alpha}{\theta^\alpha}\left(\int^\infty_0 s^{\alpha-1}f_I(s)ds\right)$.<br>
From the above, it is clear that for the Weibull distribution and after a choice of $f_I$, the compatibility condition is freely parameterized by $\alpha$ using the above relation to define $\theta(\alpha)$. 

#### $R_0$ posterior distributions
The $R_0$ value is given by $R_0 = \int^\infty_0 S_\gamma(t)\beta(t)dt$, where $S_\gamma(t)$ is the survival function of $\gamma$ and $\beta$ is the contact interval hazard. For a Weibull distribution, Wikipedia says the survival function is $S_\gamma(t) = e^{-(t/\gamma_\theta)^{\gamma_\alpha}}$. Combining with the definition of $\beta$ we obtain <br>
$R_0 = \int^\infty_0 e^{-(t/\gamma_\theta)^{\gamma_\alpha}}\frac{\beta_\alpha}{\beta_\theta}\left(\frac{t}{\beta_\theta}\right)^{\beta_\alpha-1}dt.$ <br>
If we now do a u-substition: $u = (t/\gamma_\theta)^{\gamma_\alpha}\Rightarrow t=\gamma_\theta u^{1/\gamma_\alpha}$ and $dt = \frac{\gamma_\theta}{\gamma_\alpha} u^{1/\gamma_\alpha-1}du$. Combining we have <br>
$R_0 = \int^\infty_0 e^{-u}\frac{\beta_\alpha}{\beta_\theta}\left(\frac{\gamma_\theta u^{1/\gamma_\alpha}}{\beta_\theta}\right)^{\beta_\alpha-1}\frac{\gamma_\theta}{\gamma_\alpha}u^{1/\gamma_\alpha-1}du = \frac{\beta_\alpha{\gamma_\theta}^{\beta_\alpha}}{{\beta_\theta}^{\beta_\alpha}\gamma_\alpha}\int^\infty_0 e^{-u}u^{\beta_\alpha/\gamma_\alpha - 1}du$.<br>
When does $e^{-s}s^r = e^{-s/2}$ on $\left[0,\infty\right)$? Iff $e^{-s/2}s^r=1$ iff $s^r=e^{s/2}$. You can know that if $s>2r$ and the exponential is above the poly, then it will forever thereafter be above. That's because $\partial_s\left(e^{s/2}\right) = y_1/2$ where $\partial_s\left(s^r\right)=ry_2/s.$ And so $y_1\geq y_2\Rightarrow y_1/2\geq ry_1/s\geq ry_2/s$, which means the derivative will always occur with faster rate. <br> <br>
Thus, you can grow a box until you reach this condition, then bound that tail by $e^{-s/2}$ by an exact integral for an error estimate, while using a numerical integral to compute the first part.

In [ ]:
function R₀(β_α::Real,β_θ::Real,γ_α::Real,γ_θ::Real;tol::Real=1e-6,npts::Integer=10001)
   # Find L₀ where tail of integral is at most tol when the tail is bounded by exp(-s/2)
   # You want I<tol*β_θ^β_α*γ_α/β_α/γ_θ^β_α
   # d/dx F = exp(-s/2) for F = -2 exp(-s/2); so I = 2 exp(-s/2)
    L₀ = -2*log(0.5*tol*β_θ^β_α*γ_α/β_α/γ_θ^β_α);
    L₀ = L₀>1 ? L₀ : 1.0;
    
    # Find where in [⋅>2r], the exponential is above the poly and may dominate with exp(-s/2)
    r = β_α/γ_α-1;
    L₀ = L₀ > 2*r ? L₀ : 2*r;
    while r*log(L₀) > L₀/2
        L₀ *= 2;
    end
    
    # Compute definite integral on ∫^L₀_0 by trapezoidal rule
    xs = LinRange(0,L₀,npts); Δ = xs[2];
    ys = β_α*γ_θ^β_α/β_θ^β_α/γ_α*exp.(-xs).*(xs.^r);
    
    val = [Δ*(ys[i]+ys[i-1])/2 for i=2:length(xs)];
    
    return sum(val)
end;

In [ ]:
marginals[:R₀] = [R₀(marginals[:βα][i],marginals[:βθ][i],
                     marginals[:γα][i],marginals[:γθ][i]) for i=1:length(marginals[:βα])];
pltR₀ = histogram(marginals[:R₀],normalize=:pdf,labels="",title="R₀",size=(450,250),bins=15,linecolor=:white,linewidth=0)
plot!(xlabel="value",ylabel="pdf",
      size=(figwdth/2*λplot,fighght*λplot),
      tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
      titlefontsize=round(figtitlefontsize*λfont))

In [ ]:
savefig("postR0.pdf")

In [ ]:
lay = @layout [a b];
plot!(pltR₀,ylabel="");
plot(pltβγμ,pltR₀,layout=lay,
      size=(figwdth*λplot,fighght*λplot),
      tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
      titlefontsize=round(figtitlefontsize*λfont))

In [ ]:
savefig("mst_meanR0.pdf");

## Pearson correlation coefficients

In [ ]:
μs = Dict{Symbol,Float64}(); σs = Dict{Symbol,Float64}();
for key in keys(marginals)
    μs[key] = sum(marginals[key])/length(marginals[key]);
    σs[key] = √( sum((marginals[key].-μs[key]).^2)/length(marginals[key]) );
end
n=length(keys(marginals));
PC = Matrix{Float64}(undef,n,n); id₁=0;
for key₁ in keys(marginals)
    id₁+=1;
    id₂=0;
    for key₂ in keys(marginals)
        id₂+=1;
        PC[id₁,id₂] = sum( (marginals[key₁].-μs[key₁]).*(marginals[key₂].-μs[key₂]) )/(σs[key₁]*σs[key₂]);
        PC[id₁,id₂] *= 1/length(marginals[key₁]);
    end
end;

In [ ]:
dfpc = DataFrame("prm"=>[key for key in keys(marginals)]);
pos = 0;
for key in keys(marginals)
    pos += 1;
    dfpc[!,string(key)] = convert(Vector,PC[:,pos]);
end
println("Pearson correlation coefficients (remember βθ is not free):")
dfpc

# Analyze Best Fit

## Inputs

In [ ]:
nnd = 2500;; nndsmp = 2500;
atol = 1e-5;
rtol = 1e-3; 

## Simulate the best fit

In [ ]:
df[!,1] = Symbol.(df[!,1])

# Find best fit
idℓerr = 1;
while df[idℓerr,1]!=:ℓerr
    idℓerr+=1;
end

ℓs = [df[idℓerr,k] for k=2:ncol(df)]
ℓbf = minimum(ℓs);
pos = findfirst(ℓs.==ℓbf) + 1;

In [ ]:
# Prepare the data vector
vkeys = df[:,1]; prm,_=rdprm(df[:,pos],vkeys);

# Set the input discretizations
prm[:nnd][1] = nnd; prm[:atol][1] = atol; prm[:rtol][1] = rtol; prm[:nndsmp][1] = nndsmp;
data!(prm);

println("The best fit parameters are ",prm)

In [ ]:
# Run the simulation
ysol,yʳsol = pdesolve(;prm=prm);

## Plot equation coefficients

In [ ]:
plot(:α;prm=prm)

In [ ]:
plot(:β;prm=prm)

In [ ]:
plot(:γ;prm=prm)

In [ ]:
plot(:Weibull;prm=prm)

In [ ]:
plot(:fˢ;prm=prm)

In [ ]:
plot(:fⁱ;prm=prm)

## Plot best fit solution

In [ ]:
plot(ysol)

In [ ]:
plot(ysol,yʳsol;prm=prm)

In [ ]:
plotbd(ysol;prm=prm)

### Plot errors with ODH

In [ ]:
df_yⁱ = CSV.read("ODH_snipdaily.csv",DataFrame);
first(df_yⁱ,3)

# Copied from abcmc's ℓerr
npts = nrow(df_yⁱ);
taxis = [ysol[i].yˢ.tlvl.t₀[1] for i=1:length(ysol)];

yⁱ_daily = Vector{Float64}(undef,npts);
#  Total infections during this period
kT = sum(df_yⁱ[!,"daily_confirm"]);
#  ∫yⁱdt at s=0 by trapezoidal rule
∫yⁱdt = 0.0;
@inbounds for k=2:length(taxis)
    ∫yⁱdt += (ysol[k].yⁱ.ys[1]+ysol[k-1].yⁱ.ys[1])/2*(taxis[k]-taxis[k-1]);
end
neff = kT/∫yⁱdt;

# Compute difference in daily incidence between model prediction and observed
@inbounds for k=1:npts
    tnow = k-1.0;
    ℓ = myfindfirst(taxis,tnow);
    ℓ = ℓ==1 ? 2 : ℓ;
    ynow = myinterp(tnow,ysol[ℓ-1].yⁱ,ysol[ℓ].yⁱ);
    yⁱ_daily[k] = neff*eval(ynow,0.0);
end 

In [ ]:
# Extract error for this trajectory
rms,yⁱ_daily = ℓerr(ysol;prm=prm);

# Compute 95% quantiles
qlow = [quantile(dftrjabc[i,2:end],0.0001) for i=1:nrow(dftrj)];
qhgh = [quantile(dftrjabc[i,2:end],0.9999) for i=1:nrow(dftrj)];

In [ ]:
npts = nrow(df_yⁱ);
plot(df_yⁱ[!,:time],df_yⁱ[!,:daily_confirm],linewidth=1,labels="ODH raw",linestyle=:dash,
     title="ODH Daily Cases",xlabel="date",ylabel="count",size=(400,225));
odhavg = Vector{Float64}(undef,npts);
@inbounds for i=1:npts
    Σ=0.0; ct = 0;
    for k=-3:3
        try
            Σ += df_yⁱ[i+k,:daily_confirm];
            ct += 1;
        catch
            true;
        end
    end
    odhavg[i] = Σ/ct;
end
plot!(df_yⁱ[!,:time],odhavg,linewidth=3,labels="ODH 7 day mvavg");
plot!(df_yⁱ[!,:time],yⁱ_daily[1:npts],linewidth=3,labels="model",
                     ribbon=(yⁱ_daily[1:npts]-qlow,qhgh-yⁱ_daily[1:npts]),
                     linestyle=:dashdot)
plot!(legend=:bottomright)
plot!(size=(figwdth*0.75*λplot,fighght*λplot),
      tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
      titlefontsize=round(figtitlefontsize*λfont))

In [ ]:
savefig("abcvar.pdf");